In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import yaml
import json
import time
from datetime import datetime
import multiprocessing as mp
import statsmodels.api as sm
from functools import partial
# import dolphindb as ddb
from itertools import chain, product 
from threadpoolctl import threadpool_limits 
from sklearn.linear_model import LinearRegression
import asyncpg
# from PNL import *
from flib.utils import read_supplementary_data, read_target_return, transform_column_type
from flib import DailyReader
from utils import *
import flib
import random
# pd.set_option('display.max_rows', None)  # 显示所有行
# from PNL import PNL_STOCK
warnings.filterwarnings("ignore")
# from pilot import *
from tqdm import tqdm
from implied_vol_solver import CDSImpliedVolatilitySolver

In [3]:
path_lst = os.listdir('/home/yicheng/credit/data')

In [4]:
df_all = pd.DataFrame()
for path  in tqdm(path_lst):
    a = pd.read_parquet(f'/home/yicheng/credit/data/{path}')
    use_lst = ['ticker','date','tenor','parspread','convspreard','upfront', 'runningcoupon','primarycoupon',]
    b = a.loc[a['tenor']=='5Y']
    temp = b[use_lst]
    # temp['path'] = path
    df_all = pd.concat([df_all,temp])



100%|██████████| 5/5 [00:00<00:00, 12.57it/s]


In [5]:
a.columns.tolist()

['ticker',
 'date',
 'batch',
 'shortname',
 'redcode',
 'sector',
 'region',
 'country',
 'avrating',
 'impliedrating',
 'tier',
 'currency',
 'docclause',
 'primarycurve',
 'primarypricetype',
 'runningcoupon',
 'primarycoupon',
 'tenor',
 'parspread',
 'convspreard',
 'upfront',
 'cdsrealrecovery',
 'cdsassumedrecovery',
 'carriedforward',
 'compositedepth5y',
 'primarydepthcontributed',
 'primarydepthpassed',
 'primarydepthpassed1wma',
 'primarydepthpassed1mma',
 'primarydepthpassed2mma',
 'compositepricerating',
 'compositecurverating',
 'hasclearsubmissions',
 'hasquotes',
 'hasbankcontributions',
 'clearingsubmissionsdepthcontr',
 'quotesdepthcontr',
 'bankcontributionsdepthcontr',
 'clearingsubmissionsdepthpassed',
 'quotesdepthpassed',
 'bankcontributionsdepthpassed',
 'dailyupdatefrequency',
 'maxdailymovement',
 '_52wkhigh5y',
 '_52wklow5y',
 'contrstddevpassedrecovery',
 'contrlqrangepassed',
 'contrlqrangepassedrecovery',
 'contr10thpctlpassed',
 'contr10pctlpassedrecovery

In [6]:
df_use = df_all.sort_values(by=['date',],inplace=False).groupby('date').first()

In [7]:
def read_financial_data(field_name, table_name ,start_date, end_date):
    """
    Read financial data from DolphinDB database.
    
    Args:
        field_name (str): The name of the field to read.
        start_date (str): The start date of the data to read.
        end_date (str): The end date of the data to read.
        
    Returns:
    """ 
    data = DailyReader(rf'sp_financials_{table_name}').get_data(field_name, column_type='tradingitemid').loc[start_date:end_date]
    return data

In [8]:
long_term_debt = read_financial_data('financials_1049_ytd', 'ytd', '2020-01-01', '2025-01-01')
current_asset = read_financial_data('financials_1008_ytd', 'ytd', '2020-01-01', '2025-01-01')
current_asset_short_term_ratio = read_financial_data('financials_43901_ytd', 'ytd', '2020-01-01', '2025-01-01')
shares = read_financial_data('financials_1070_ytd', 'ytd', '2020-01-01', '2025-01-01')
short_term_debt = current_asset / current_asset_short_term_ratio
financial_debt = long_term_debt + short_term_debt
financial_debt_ratio = financial_debt / shares

In [9]:
close = DailyReader('sp_stock_daily_2010').get_data('close')
close_ticker = transform_column_type(df = close, column_type='ticker')
financial_debt_ratio = transform_column_type(df = financial_debt_ratio, column_type='ticker')
df_use.index = pd.to_datetime(df_use.index)

# 使用reindex替代loc，自动处理缺失索引
close_ticker_aligned = close_ticker.reindex(df_use.index)
financial_debt_aligned = financial_debt_ratio.reindex(df_use.index)
close_ticker_aligned['XRX'].name = 'close_price'
financial_debt_aligned['XRX'].name = 'financial_debt_ratio'


In [10]:
df_use

,ticker,tenor,parspread,convspreard,upfront,runningcoupon,primarycoupon
date,,,,,,,
2021-01-01,XRX,5Y,0.017344,0.016782,-0.154914,0.05,N
2021-01-04,XRX,5Y,0.017354,0.016799,-0.154583,0.05,N
2021-01-05,XRX,5Y,0.017326,0.016759,-0.154758,0.05,N
2021-01-06,XRX,5Y,0.017109,0.017250,0.033659,0.01,Y
2021-01-07,XRX,5Y,0.016987,0.017125,0.033055,0.01,Y
...,...,...,...,...,...,...,...
2025-09-08,XRX,5Y,0.212217,0.253803,0.397754,0.05,N
2025-09-09,XRX,5Y,0.218331,0.264765,0.407366,0.05,N
2025-09-10,XRX,5Y,0.224795,0.276327,0.416382,0.05,N


In [11]:
df = pd.concat([df_use,close_ticker_aligned['XRX'],financial_debt_aligned['XRX']],axis=1).dropna()

In [12]:
df.to_csv('res/input_data.csv')

In [12]:
import sys
from io import StringIO
from contextlib import redirect_stdout
import logging
logging.getLogger('implied_vol_solver').setLevel(logging.WARNING)  # 只显示警告和错误

# S = 100.0        # 资产价值
D = 150.0        # 债务面值  
t = 5.0          # CDS期限
r = 0.05         # 无风险利率
R = 0.4          # 回收率
L = 0.5          # 债务回收率
lamb = 0.3       # 违约壁垒参数
from tqdm import tqdm
# 先创建新列
df['implied_vol'] = np.nan 
df['par_spread_error'] = np.nan

with redirect_stdout(StringIO()):
    for i, row in tqdm(df.iterrows()):
        # 参数设置

        solver = CDSImpliedVolatilitySolver(row['close_price'], row['financial_debt_ratio'], t, r, L, lamb)

        # 从市场CDS价差反解波动率
        market_cds_spread = row['parspread']  # 250bp
        implied_vol,error = solver.solve_implied_volatility(market_cds_spread, R, method='brent');
        df.at[i, 'implied_vol'] = implied_vol
        df.at[i, 'par_spread_error'] = error
        # break

    # print(f"隐含波动率: {implied_vol:.6f}")

1003it [00:05, 189.13it/s]


In [17]:
today = datetime.now().strftime('%Y%m%d')
df.to_csv(f'res/res_{today}.csv')

In [16]:
df.implied_vol.describe()

count    1003.000000
mean        0.268293
std         0.032430
min         0.177074
25%         0.248491
50%         0.280670
75%         0.292002
max         0.317479
Name: implied_vol, dtype: float64

## Draft Files

In [19]:
fin_ytd_info = read_supplementary_data('stock', 'financials_ytd_dataitem')
fin_ltm_info = read_supplementary_data('stock', 'financials_ltm_dataitem')
fin_info = pd.concat([fin_ytd_info, fin_ltm_info], axis=0)
fin_info

,Table name,Defination,Dataitemid,Type,Source
0,financials_1_ytd,Cost Of Revenues,1,ytd,Income Statement
1,financials_2_ytd,"Depreciation & Amortization, Total - (IS)",2,ytd,Income Statement
2,financials_4_ytd,"EBT, Excl. Unusual Items",4,ytd,Income Statement
3,financials_6_ytd,Earnings Before Xtra Ord. Item & Accounting Ch...,6,ytd,Income Statement
4,financials_7_ytd,Earnings From Continuing Operations,7,ytd,Income Statement
...,...,...,...,...,...
653,financials_49175_ltm,"Net Debt/EBITDA, Excl. Operating Lease Adjustment",49175,ltm,Calculated Intra-Period Ratio
654,financials_49182_ltm,"Net Debt/(EBITDA, Excl. Operating Lease Adjust...",49182,ltm,Calculated Intra-Period Ratio
655,financials_49183_ltm,"Total Debt/(EBITDA, Excl. Operating Lease Adju...",49183,ltm,Calculated Intra-Period Ratio
656,financials_100689_ltm,EBITA,100689,ltm,Income Statement


In [20]:
fin_info.loc[fin_info['Table name'].isin(['financials_1049_ytd','financials_1008_ytd','financials_43901_ytd','financials_1070_ytd'])]

,Table name,Defination,Dataitemid,Type,Source
63,financials_1008_ytd,Total Current Assets,1008,ytd,Balance Sheet
77,financials_1049_ytd,Long-Term Debt,1049,ytd,Balance Sheet
85,financials_1070_ytd,Total Common Shares Outstanding,1070,ytd,Balance Sheet Shares
600,financials_43901_ytd,Current Assets / Short Term Debt,43901,ytd,Calculated Intra-Period Ratio


In [15]:
def read_financial_data(field_name, table_name ,start_date, end_date):
    """
    Read financial data from DolphinDB database.
    
    Args:
        field_name (str): The name of the field to read.
        start_date (str): The start date of the data to read.
        end_date (str): The end date of the data to read.
        
    Returns:
    """ 
    data = DailyReader(rf'sp_financials_{table_name}').get_data(field_name, column_type='tradingitemid').loc[start_date:end_date]
    return data

In [16]:
long_term_debt = read_financial_data('financials_1049_ytd', 'ytd', '2020-01-01', '2025-01-01')
current_asset = read_financial_data('financials_1008_ytd', 'ytd', '2020-01-01', '2025-01-01')
ratio = read_financial_data('financials_43901_ytd', 'ytd', '2020-01-01', '2025-01-01')
shares = read_financial_data('financials_1070_ytd', 'ytd', '2020-01-01', '2025-01-01')
short_term_debt = current_asset / ratio
financial_debt = long_term_debt + short_term_debt
financial_debt_ratio = financial_debt / shares

In [17]:
financials_1049_ytd
financials_1008_ytd	current asset
financials_43901_ytd ratio


SyntaxError: invalid syntax (2822487968.py, line 2)

In [ ]:
a = pd.read_parquet('/home/yicheng/credit/data/mkidwsfpi2qmdzop.parquet')
use_lst = ['ticker','date','tenor','parspread','upfront']
b = a.loc[a['tenor']=='5Y']

In [ ]:
set(use_lst) - set(b.columns)

set()

In [ ]:
b[use_lst]

,ticker,date,tenor,parspread,upfront
162,XRX,2021-01-01,5Y,0.017650,0.036259
163,XRX,2021-01-01,5Y,0.017650,-0.153332
164,XRX,2021-01-01,5Y,0.017812,0.037005
165,XRX,2021-01-01,5Y,0.017812,-0.152473
166,XRX,2021-01-01,5Y,0.017812,0.037005
...,...,...,...,...,...
63241,XRX,2021-12-31,5Y,0.020569,-0.135467
63242,XRX,2021-12-31,5Y,0.020569,0.048645
63243,XRX,2021-12-31,5Y,0.020569,-0.135467
63244,XRX,2021-12-31,5Y,0.020569,0.048645


In [ ]:
reader = DailyReader('option_factors_henry')

In [ ]:
reader.field_idx

['stock_tot_volume',
 'stock_tot_amount',
 'so_vol_ratio',
 'log_so_vol_ratio',
 'option_tot_volume',
 'pcratio',
 'tot_open_interest',
 'iv_slope',
 'skewiv',
 'minute_rv',
 'daily_rv',
 'C_1_A_iv',
 'C_1_B_iv',
 'C_1_C_iv',
 'C_1_D_iv',
 'C_1_E_iv',
 'C_1_F_iv',
 'C_1_G_iv',
 'C_1_H_iv',
 'C_1_I_iv',
 'C_1_J_iv',
 'C_1_K_iv',
 'C_1_L_iv',
 'C_1_M_iv',
 'C_1_N_iv',
 'C_1_O_iv',
 'C_1_P_iv',
 'C_1_Q_iv',
 'C_2_A_iv',
 'C_2_B_iv',
 'C_2_C_iv',
 'C_2_D_iv',
 'C_2_E_iv',
 'C_2_F_iv',
 'C_2_G_iv',
 'C_2_H_iv',
 'C_2_I_iv',
 'C_2_J_iv',
 'C_2_K_iv',
 'C_2_L_iv',
 'C_2_M_iv',
 'C_2_N_iv',
 'C_2_O_iv',
 'C_2_P_iv',
 'C_2_Q_iv',
 'C_3_A_iv',
 'C_3_B_iv',
 'C_3_C_iv',
 'C_3_D_iv',
 'C_3_E_iv',
 'C_3_F_iv',
 'C_3_G_iv',
 'C_3_H_iv',
 'C_3_I_iv',
 'C_3_J_iv',
 'C_3_K_iv',
 'C_3_L_iv',
 'C_3_M_iv',
 'C_3_N_iv',
 'C_3_O_iv',
 'C_3_P_iv',
 'C_3_Q_iv',
 'P_1_A_iv',
 'P_1_B_iv',
 'P_1_C_iv',
 'P_1_D_iv',
 'P_1_E_iv',
 'P_1_F_iv',
 'P_1_G_iv',
 'P_1_H_iv',
 'P_1_I_iv',
 'P_1_J_iv',
 'P_1_K_iv',
 'P_1_L